<a href="https://colab.research.google.com/github/MarcoDiFrancesco/SmartElevator/blob/main/costruzione%20dataframe%20finale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#MOUNTING DEL DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#APERTURA FILE
import json
with open("/content/drive/My Drive/SmartElevator/28-09_07-11_alldata.json", "r") as f:
    data = json.load(f)

In [3]:
#OPERAZIONI DATAFRAME
import pandas as pd
pd.DataFrame(data)
dfs = []
for result in data["results"]:
    serie = result["series"][0]
    for serie in result["series"]:
        df = pd.DataFrame(serie["values"], columns=serie["columns"])
        df.name = serie["name"]
        dfs.append(df)

In [4]:
df = dfs[0]
df_temp = dfs[2]
df_curr = dfs[3]
df = df.drop([
    'client', "GW", "device", "mileage", "vibra_custom",
    'vib_x_root', 'vib_y_root', 'vib_z_root',
], axis=1)
df_temp = df_temp.drop(['client', "GW", "device", 'weekday'], axis=1)
df_curr = df_curr.drop(['client', "GW", "device", 'weekday'], axis=1)

In [5]:
#JOIN DATAFRAME
df = df.set_index('time')
df = df.rename(columns={'temp': 'temperature_surface'})
df_temp = df_temp.set_index('time').rename(columns={"value": "temperature_external"})
df_curr = df_curr.set_index('time').rename(columns={"value": "current"})

df = df.join(df_temp)
df = df.join(df_curr)

df = df.reset_index()
df["time"] = pd.to_datetime(df["time"], unit='ms')

In [6]:
df.loc[(df["time"] >= "2021-10-09") & (df["time"] <= "2021-10-12"), "label"] = "working-engine-1"
df.loc[(df["time"] >= "2021-10-27 17:30") & (df["time"] <= "2021-10-29 00:00"), "label"] = "bearings-1"

df.loc[(df["time"] >= "2021-11-03 16:00") & (df["time"] <= "2021-11-05 11:00"), "label"] = "working-engine-2" # Every 2 minutes (TOCHECK)
df.loc[(df["time"] >= "2021-11-05 13:30") & (df["time"] <= "2021-11-05 23:59"), "label"] = "bearings-2" # Every 2 minutes (TOCHECK)

In [7]:
df = df.drop('temperature_surface', 1) #questa conviene eliminarla dato che hanno detto è irrilevante
df=df.dropna()

In [8]:
import numpy as np

#CREAZIONE COLONNE LABELS
#per i bearings
conditions=[
    (df["label"]=="bearings-1") | (df["label"]=="bearings-2"),
    (df["label"]=="working-engine-1") | (df["label"]=="working-engine-2")
    ]
values = [1, 0]
df["bearings"]=np.select(conditions, values)

#per il resto appena abbiamo i dati bisogna aggungere la riga
df["rotor"]=0
df["electricity"]=0

In [12]:
#CREAZIONE COLONNA GIORNO E MACCHINA
df["day"]=0
df["machine"]=0
counter_measure=0
counter_day=0 
counter_machine=0
for i, row in df.iterrows():
  print(i)
  if ((counter_measure%50)==0): #i 50 giorni sono in un piccolo condominio. sto approfondendo con christian per avere un caso medio
    counter_measure=0
    counter_day+=1
  if ((counter_day%7)==0):#i 7 giorni sono arbitrari ma penso siano opportuni dato che comunque sono dati appartenenti al medesimo motore in ciclo giornaliero
    counter_measure=0
    counter_day=0
    counter_machine+=1
  df.at[i, 'day'] = counter_day
  df.at[i, 'machine'] = counter_machine
  counter_measure+=1


3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
3225
3226
3227
3228
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249


In [13]:
df=df.drop("label", 1)
df

,time,s_00,s_01,s_02,s_03,s_04,s_05,s_06,s_07,s_08,s_09,s_10,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,sonic_custom,sonic_rmslog,vib_x_acc,vib_x_f1,vib_x_f2,vib_x_f3,vib_x_kurt,vib_x_peak,vib_x_vel,vib_y_acc,vib_y_f1,vib_y_f2,vib_y_f3,vib_y_kurt,vib_y_peak,vib_y_vel,vib_z_acc,vib_z_f1,vib_z_f2,vib_z_f3,vib_z_kurt,vib_z_peak,vib_z_vel,temperature_external,current,label,bearings,rotor,electricity,day,machine
3050,2021-10-09 00:00:47.249,-0.068780,-4.477798,-6.905241,-4.547951,-2.283589,-0.124857,-1.274090,-0.499428,-1.128634,-0.117876,0.031930,0.127718,-1.477684,1.652895,-1.119364,-1.066148,-0.050813,-0.190318,-0.391394,-0.188945,0.987747,1.496910,1.109608,0.488277,0.347365,0.583524,0.352694,0.484375,1.719167,1.341077,2.497537,2.766388,1.760176,0.521458,0.685517,2.844896,1.684042,1.109990,2.106870,0.603318,0.362336,0.576215,2.100146,21.507958,3.967956,working-engine-1,0,0,0,1,0
3051,2021-10-09 00:05:47.003,0.368391,-5.471160,-5.840925,-3.531701,-1.571756,-1.326505,-1.415999,0.166629,-0.261158,-0.461204,1.348020,0.178073,-1.223621,1.787938,-2.060082,-0.960861,0.029297,-0.066720,-0.322728,0.156672,0.944885,1.458000,1.150203,0.454436,0.111863,0.251716,0.371477,0.553571,1.753871,1.351637,0.403941,0.404274,0.506051,0.530757,0.740439,2.838546,1.700080,4.016145,6.259542,4.072398,0.352717,0.611753,2.122022,21.507958,3.950103,working-engine-1,0,0,0,1,0
3052,2021-10-09 00:10:46.911,0.063973,-5.670291,-6.273518,-3.701076,-2.121080,-0.683337,-1.354200,-0.224765,-0.041428,-0.458915,0.425612,0.306249,-0.445411,1.636873,-1.961662,-0.477913,0.418402,-0.265850,0.105287,0.298581,0.964267,1.595331,1.136671,2.233503,2.702384,2.951945,0.379042,0.542411,1.721837,1.362196,1.113300,1.334103,1.760176,0.520424,0.724166,2.835371,1.668003,7.346115,10.625954,4.796380,0.353037,0.573677,2.068546,21.507958,3.950103,working-engine-1,0,0,0,1,0
3053,2021-10-09 00:15:47.253,-0.116846,-5.862554,-6.252918,-3.568322,-2.457542,-0.948844,-0.878119,-0.618448,0.189746,0.184253,0.752918,-0.037079,-0.932937,1.943580,-1.794575,-0.958572,-0.419318,-0.474136,-0.135042,-0.486496,0.887222,1.499199,1.123139,1.503505,1.760377,1.212815,0.349825,0.493304,1.719167,1.362196,0.384236,0.415824,0.242024,0.523180,0.756713,2.863947,1.684042,5.206862,7.816794,4.404223,0.358809,0.581292,2.100146,21.507958,3.960632,working-engine-1,0,0,0,1,0
3054,2021-10-09 00:20:46.707,0.256237,-5.418517,-6.527581,-3.382925,-1.855573,-0.443008,-0.189174,-0.153811,-1.114900,0.147631,1.686770,0.230716,-0.981003,1.575074,-1.858663,-0.949416,-0.451362,-0.007210,-0.151064,0.623598,1.054281,1.320668,1.150203,2.480058,2.884898,2.276888,0.398085,0.575893,1.807261,1.383316,0.837438,0.421600,0.275028,0.539023,0.775020,2.971900,1.716119,4.904137,7.267176,3.499246,0.354320,0.576215,2.185221,21.507958,3.962463,working-engine-1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12554,2021-11-05 23:50:15.978,-1.482695,2.671574,-1.741697,-0.528364,0.151667,0.110949,0.095409,-0.092759,-0.193348,0.070954,-3.601932,-2.806857,2.431605,2.846370,2.641457,3.359314,-0.721953,-0.235752,0.169616,-0.035658,1.352331,2.821554,1.229548,1.000000,1.000000,1.000000,0.950835,1.142905,1.841410,0.855000,0.000000,0.000000,0.000000,1.078244,0.974865,1.260706,1.507937,1.000000,1.000000,1.000000,0.962243,1.384422,2.884337,20.155202,3.953765,bearings-2,1,0,0,0,2527
12555,2021-11-05 23:52:15.932,-1.370541,1.776632,-1.185506,0.066738,-0.184795,0.005662,0.619556,-0.637506,-0.152149,0.059510,-2.745901,-2.294154,4.873812,3.070677,4.204744,5.996073,0.124923,-0.256352,0.409946,0.071918,1.319985,2.963463,1.229548,1.000000,1.000000,1.000000,1.057295,1.307079,1.807930,0.827857,0.000000,0.000000,0.000000,1.033162,1.014333,1.211529,1.470238,1.000000,1.000000,1.000000,0.924884,1.222111,2.731727,20.155202,3.949187,bearings-2,1,0,0,0,2528
12556,2021-11-05 23:54:15.878,-1.981665,1.714833,-1.2